In [1]:
import os
import numpy as np
import pandas as pd
import itertools

In [2]:
top_models_path = [
    'data/non_aggregated_results_fairautoml_adult_flipped_20200903.csv',  # Adult
    'data/non_aggregated_results_fairautoml_AOF_20200903.csv',            # AOF
    'data/non_aggregated_results_fairautoml_compas_20200904.csv',         # COMPAS
    'data/non_aggregated_results_fairautoml_donors_choose_20200902.csv',  # Donors Choose
    'data/non_aggregated_results_fairautoml_oob_min_metric.csv',          # OOB Tuners (TPE)
]

In [3]:
results_df = None

for path in top_models_path:#[:4]:
    df = pd.read_csv(path)
    
    perf_metric = df['performance_metric'].unique()[0]
    fair_metric = df['fairness_metric'].unique()[0]
    
    if 'performance_val' not in set(df.columns):
        df['performance_val'] = df[f"validation_{perf_metric}"]
    if 'fairness_val' not in set(df.columns):
        df['fairness_val'] = df[f"validation_{fair_metric}"]
    if 'performance_test' not in set(df.columns):
        df['performance_test'] = df[f"test_{perf_metric}"]
    if 'fairness_test' not in set(df.columns):
        df['fairness_test'] = df[f"test_{fair_metric}"]
    
    results_df = df if results_df is None else pd.concat((results_df, df), axis=0)

In [4]:
results_df = results_df[[
    'dataset', 'task_type', 'alpha',
    'performance_val', 'fairness_val',
    'performance_test', 'fairness_test',
    'model_classpath', 'hyperparameters', 'class_ratio',
    'model_config_uuid', 'model_uuid',
    'si', 'i', 'ni', 'ri', 'target_threshold',
]]

In [5]:
results_df.head()

,dataset,task_type,alpha,performance_val,fairness_val,performance_test,fairness_test,model_classpath,hyperparameters,class_ratio,model_config_uuid,model_uuid,si,i,ni,ri,target_threshold
0,Adult,Hyperband,dynamic,0.94100,0.91024,0.93272,0.84107,fairautoml.nn.wrappers.FeedForwardClassifier,"{'batch_size': 2048, 'max_epochs': 75, 'optimi...",NaN,e90d361bb0bfc5448ae077a02f47afe2,98f05504756f5a66e28f5515f4c00cdc,3.0,0.0,34.0,3.704,tpr = 50
1,Adult,Hyperband,0.5,0.91896,0.99948,0.91367,0.94126,fairautoml.nn.wrappers.FeedForwardClassifier,"{'batch_size': 2048, 'max_epochs': 25, 'optimi...",NaN,ffa8b120ec9b1ae9201a6634fd9dffa4,cad876af4952ec67447a5e79422310c2,4.0,0.0,81.0,1.235,tpr = 50
2,Adult,Hyperband,dynamic,0.93142,0.93815,0.92774,0.88856,fairautoml.nn.wrappers.FeedForwardClassifier,"{'batch_size': 2048, 'max_epochs': 25, 'optimi...",NaN,592dde8641d47d278a158df7d45f1ce0,772f5373cdb06dc996a79f8474acef54,3.0,0.0,34.0,3.704,tpr = 50
3,Adult,Hyperband,dynamic,0.94315,0.93658,0.93605,0.88501,fairautoml.nn.wrappers.FeedForwardClassifier,"{'batch_size': 2048, 'max_epochs': 25, 'optimi...",NaN,7e1266b6f483aaa91345b090acfa8e87,8a3f2ed18191d2b036bf6d656fb63891,3.0,1.0,11.0,11.111,tpr = 50
4,Adult,Hyperband,dynamic,0.92067,0.99373,0.91778,0.93524,fairautoml.nn.wrappers.FeedForwardClassifier,"{'batch_size': 2048, 'max_epochs': 25, 'optimi...",NaN,25b7ae7ed998f0b159e77417ec8aab68,cbbaad53880e7dace0db138a34868a0c,4.0,0.0,81.0,1.235,tpr = 50


In [6]:
results_df.value_counts('dataset')

dataset
AOF              135
Adult            135
COMPAS           135
Donors Choose    135
dtype: int64

In [7]:
results_df = results_df[results_df['task_type'] != 'RandomSampler']

In [8]:
results_df['tuner'] = results_df.apply(
    lambda row: f"{row.task_type} {row.alpha}",
    axis=1,
)

# Changing to naming convention used on the paper
tuner_rename = {
    'RandomSearch 0.5': 'FairRS',
    'RandomSearch 1': 'RS',
    'Hyperband dynamic': 'FB-auto',
    'Hyperband 0.5': 'FB',
    'Hyperband 1': 'HB',
    'TPESampler 0.5': 'FairTPE',
    'TPESampler 1.0': 'TPE',
}

results_df['tuner'] = results_df['tuner'].apply(
    lambda tuner: tuner_rename[tuner] if tuner in tuner_rename else tuner
)
results_df['tuner'].value_counts()

FB-auto    60
FB         60
RS         60
TPE        60
FairTPE    60
FairRS     60
HB         60
Name: tuner, dtype: int64

In [9]:
key_columns = ['dataset', 'tuner', 'alpha']
metric_columns = ['performance_val', 'fairness_val', 'performance_test', 'fairness_test']

results_df_metrics = results_df[key_columns + metric_columns]

agg_results = results_df_metrics.groupby(['dataset', 'tuner', 'alpha']).agg(['mean', 'std'])
agg_results.round(decimals=3) * 100

performance_val       fairness_val        \
                                         mean   std         mean   std   
dataset       tuner   alpha                                              
AOF           FB      0.5                50.7  15.8         76.0  15.0   
              FB-auto dynamic            61.7   6.5         68.1  14.0   
              FairRS  0.5                60.4   9.2         64.1  15.6   
              FairTPE 0.5                55.7   7.1         76.9  17.0   
              HB      1                  68.4   1.0         32.3   6.2   
              RS      1                  67.8   0.6         42.2  10.7   
              TPE     1.0                68.7   1.0         30.5   4.8   
Adult         FB      0.5                92.7   3.4         94.0   5.4   
              FB-auto dynamic            92.0   4.4         94.7   2.8   
              FairRS  0.5                93.6   6.5         79.4   8.1   
              FairTPE 0.5                93.3   3.8         82.2   5.7   
              HB      1                  99.4   0.1         53.5   1.0   
              RS      1                  99.4   0.1         55.7   2.9   
              TPE     1.0                99.4   0.1         54.9   2.1   
COMPAS        FB      0.5                71.2   5.1         95.5   2.7   
              FB-auto dynamic            74.0   3.5         95.8   2.9   
              FairRS  0.5                67.4   7.6         77.4  13.6   
              FairTPE 0.5                67.1   7.6         81.8  12.4   
              HB      1                  78.1   0.3         45.4   6.8   
              RS      1                  77.7   0.3         43.8   6.9   
              TPE     1.0                78.0   0.3         42.8   7.8   
Donors Choose FB      0.5                54.2   1.4         97.7   2.1   
              FB-auto dynamic            54.2   1.6         98.2   1.7   
              FairRS  0.5                51.7   2.2         97.0   2.3   
              FairTPE 0.5                52.3   2.2         96.3   3.0   
              HB      1                  60.9   0.6         28.7   3.4   
              RS      1                  59.9   0.9         24.9   2.9   
              TPE     1.0                61.0   0.7         27.1   2.1   

                              performance_test       fairness_test        
                                          mean   std          mean   std  
dataset       tuner   alpha                                               
AOF           FB      0.5                 52.6  14.6          81.3  12.3  
              FB-auto dynamic             64.0   5.1          74.2  12.9  
              FairRS  0.5                 62.6  10.0          68.6  15.6  
              FairTPE 0.5                 59.2   9.0          80.0  14.6  
              HB      1                   68.4   1.4          35.2   6.7  
              RS      1                   68.1   1.6          45.0  11.5  
              TPE     1.0                 68.5   1.5          33.7   5.2  
Adult         FB      0.5                 92.3   3.3          89.5   6.1  
              FB-auto dynamic             91.6   4.2          90.9   5.2  
              FairRS  0.5                 93.8   6.3          78.6   8.0  
              FairTPE 0.5                 93.5   3.7          80.7   6.0  
              HB      1                   99.0   0.1          54.1   1.2  
              RS      1                   99.1   0.1          56.6   3.2  
              TPE     1.0                 99.1   0.1          55.6   2.4  
COMPAS        FB      0.5                 67.6   4.8          80.7  14.8  
              FB-auto dynamic             70.1   4.3          90.0   6.5  
              FairRS  0.5                 64.2   6.7          67.8  11.9  
              FairTPE 0.5                 63.9   6.7          69.5  14.5  
              HB      1                   73.6   1.4          51.2   7.7  
              RS      1                   73.2   0.9          43.4   5.8  
          

## Compute statistical differences between every candidate pair of results

In [10]:
from scipy import stats

results_indexed = results_df_metrics.set_index(['dataset', 'tuner'])

all_statistical_tests = dict()

for metric_col in metric_columns:
    metric_tests = dict()

    for dataset in results_df_metrics['dataset'].unique():
        dataset_results = results_indexed.loc[dataset]
        dataset_tests = dict()

        for tuner_type_outer in results_df_metrics['tuner'].unique():
            outer_tuner_tests = dict()

            outer_values = dataset_results.loc[tuner_type_outer]
            outer_values = outer_values[metric_col]

            for tuner_type_inner in results_df_metrics['tuner'].unique():

                inner_values = dataset_results.loc[tuner_type_inner]
                inner_values = inner_values[metric_col]

                # Run statistical test
                stats_val, p_val = stats.ks_2samp(outer_values, inner_values)

                outer_tuner_tests[tuner_type_inner] = p_val  # Extract p-value

            dataset_tests[tuner_type_outer] = outer_tuner_tests
            
        metric_tests[dataset] = dataset_tests

    all_statistical_tests[metric_col] = metric_tests

## Aggregate results

In [11]:
stats_tests_dfs = dict()

for metric in all_statistical_tests.keys():
    for dataset in all_statistical_tests[metric].keys():
        dataset_tests = all_statistical_tests[metric][dataset]
        tests_df = pd.DataFrame(dataset_tests)
        stats_tests_dfs[f"{dataset}_{metric}"] = tests_df


## Show p-values

In [12]:
from IPython.display import Markdown, display

for key, df in stats_tests_dfs.items():
    styled_df = df.style.applymap(
        lambda x: "background-color: #fc6203" if x<1e-2
            else "background-color: #fcba03" if x<5e-2
            else "background-color: #dffc03" if x<1e-1
            else "background-color: white"
    )

    display(Markdown(f"### {key}"))
    display(styled_df)
    print()


### Adult_performance_val

,FB-auto,FB,RS,HB,FairRS,FairTPE,TPE
FB-auto,1.000000,0.678138,0.000000,0.000000,0.001837,0.075464,0.000000
FB,0.678138,1.000000,0.000000,0.000000,0.001837,0.385547,0.000000
RS,0.000000,0.000000,1.000000,0.938331,0.000000,0.000000,0.184416
HB,0.000000,0.000000,0.938331,1.000000,0.000000,0.000000,0.184416
FairRS,0.001837,0.001837,0.000000,0.000000,1.000000,0.075464,0.000000
FairTPE,0.075464,0.385547,0.000000,0.000000,0.075464,1.000000,0.000000
TPE,0.000000,0.000000,0.184416,0.184416,0.000000,0.000000,1.000000


### AOF_performance_val

,FB-auto,FB,RS,HB,FairRS,FairTPE,TPE
FB-auto,1.000000,0.007656,0.000006,0.000000,0.678138,0.026248,0.000006
FB,0.007656,1.000000,0.000000,0.000000,0.184416,0.678138,0.000000
RS,0.000006,0.000000,1.000000,0.184416,0.000052,0.000000,0.007656
HB,0.000000,0.000000,0.184416,1.000000,0.000353,0.000000,0.678138
FairRS,0.678138,0.184416,0.000052,0.000353,1.000000,0.075464,0.000353
FairTPE,0.026248,0.678138,0.000000,0.000000,0.075464,1.000000,0.000000
TPE,0.000006,0.000000,0.007656,0.678138,0.000353,0.000000,1.000000


### COMPAS_performance_val

...

,FB-auto,FB,RS,HB,FairRS,FairTPE,TPE
FB-auto,1.000000,0.184416,0.000000,0.000000,0.385547,0.385547,0.000000
FB,0.184416,1.000000,0.000000,0.000000,0.075464,0.938331,0.000000
RS,0.000000,0.000000,1.000000,0.026248,0.000353,0.000000,0.001837
HB,0.000000,0.000000,0.026248,1.000000,0.000000,0.000000,0.938331
FairRS,0.385547,0.075464,0.000353,0.000000,1.000000,0.075464,0.000000
FairTPE,0.385547,0.938331,0.000000,0.000000,0.075464,1.000000,0.000000
TPE,0.000000,0.000000,0.001837,0.938331,0.000000,0.000000,1.000000


### COMPAS_fairness_test

,FB-auto,FB,RS,HB,FairRS,FairTPE,TPE
FB-auto,1.000000,0.184416,0.000000,0.000000,0.000006,0.000052,0.000000
FB,0.184416,1.000000,0.000000,0.000052,0.001837,0.007656,0.000052
RS,0.000000,0.000000,1.000000,0.075464,0.000006,0.000006,0.385547
HB,0.000000,0.000052,0.075464,1.000000,0.007656,0.007656,0.678138
FairRS,0.000006,0.001837,0.000006,0.007656,1.000000,0.938331,0.000052
FairTPE,0.000052,0.007656,0.000006,0.007656,0.938331,1.000000,0.000052
TPE,0.000000,0.000052,0.385547,0.678138,0.000052,0.000052,1.000000


### Donors Choose_fairness_test

,FB-auto,FB,RS,HB,FairRS,FairTPE,TPE
FB-auto,1.000000,0.938331,0.000000,0.000000,0.000353,0.000353,0.000000
FB,0.938331,1.000000,0.000000,0.000000,0.001837,0.000353,0.000000
RS,0.000000,0.000000,1.000000,0.385547,0.000000,0.000000,0.678138
HB,0.000000,0.000000,0.385547,1.000000,0.000000,0.000000,0.938331
FairRS,0.000353,0.001837,0.000000,0.000000,1.000000,0.385547,0.000000
FairTPE,0.000353,0.000353,0.000000,0.000000,0.385547,1.000000,0.000000
TPE,0.000000,0.000000,0.678138,0.938331,0.000000,0.000000,1.000000
